In [1]:
import pandas as pd
import numpy as np
import os
from pandas import Series, DataFrame
group_train = pd.read_csv("C:\\Users\\karis\\NewGroup_data.txt", sep="\t", header = None, names= ['phrase_id', 'review', 'group_id'])
group_train.head()

,phrase_id,review,group_id
0,1,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
1,2,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
2,3,['From: I3150101@dbstu1.rz.tu-bs.de (Benedikt ...,0
3,4,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
4,5,"['From: strom@Watson.Ibm.Com (Rob Strom)\n', '...",0


In [2]:
group_test = pd.read_csv("C:\\Users\\karis\\NewGroup_data_test.txt", sep="\t", header = None, names= ['phrase_id', 'review', 'group_id'])
group_test.head()

,phrase_id,review,group_id
0,1,['From: decay@cbnewsj.cb.att.com (dean.kaflowi...,0
1,2,"['From: cfaehl@vesta.unm.edu (Chris Faehl)\n',...",0
2,3,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
3,4,"['From: dps@nasa.kodak.com (Dan Schaertel,,,)\...",0
4,5,"['From: halat@panther.bears (Jim Halat)\n', 'S...",0


In [4]:
# maping cateogories to train dataframe

group_train['category'] = group_train.group_id.map({0:'alt', 1:'comp',2:'comp', 3:'comp', 4:'comp',5:'comp',6:'misc', 7:'rec', 8:'rec',9:'rec', 10:'rec', 11:'sci',
                                    12:'sci', 13:'sci', 14:'sci', 15:'religion', 
                                    16:'politics', 17:'politics', 18:'politics', 19:'religion'})

# maping cateogories to test dataframe

group_test['category'] = group_test.group_id.map({0:'alt', 1:'comp',2:'comp', 3:'comp', 4:'comp',5:'comp',6:'misc', 7:'rec', 8:'rec',9:'rec', 10:'rec', 11:'sci',
                                    12:'sci', 13:'sci', 14:'sci', 15:'religion', 
                                    16:'politics', 17:'politics', 18:'politics', 19:'religion'})

group_test.shape

(7532, 4)

In [5]:
# selecting categories 
group_train_category = group_train[(group_train.category == 'comp') | (group_train.category == 'rec') | (group_train.category == 'politics') | (group_train.category == 'religion')]
group_test_category = group_test[(group_test.category == 'comp') | (group_test.category == 'rec') | (group_test.category == 'politics') | (group_test.category == 'religion')]


In [6]:
# Concatinating Train and Test datasets

group_train_test_category = pd.concat([group_train_category, group_test_category])

In [7]:
import numpy as np
from keras.utils import np_utils

from sklearn.preprocessing import LabelEncoder

Label_encoded = LabelEncoder()
Y_label_encoded = Label_encoded.fit_transform(group_train_test_category['category'].values)
Y = np_utils.to_categorical(Y_label_encoded)
X = group_train_test_category.review


print(Y.shape)
print(X.shape)
print(Y_label_encoded)
print(Y)


Using TensorFlow backend.


(13120, 4)
(13120,)
[0 0 0 ..., 3 3 3]
[[ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 ..., 
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]]


In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

X_sequences = tokenizer.texts_to_sequences(X)

X_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(X_word_index))

X_data = pad_sequences(X_sequences, maxlen=500)

print('Shape of data tensor:', X_data.shape)



Found 172367 unique tokens.
Shape of data tensor: (13120, 500)


In [9]:
X_train =X_data[:10000]
X_test = X_data[10000:]
Y_train = Y[:10000]
Y_test =Y[10000:]

In [14]:
import gensim
import numpy as np




In [15]:
model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/karis/Desktop/latently/GoogleNews-vectors-negative300.bin', binary=True)

In [19]:
weights = model.syn0
weights.shape

(3000000, 300)

In [26]:
import numpy
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras import losses
from keras.utils import np_utils
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


In [27]:

model = Sequential()
model.add(Embedding(input_dim=weights.shape[0], output_dim=weights.shape[1], weights=[weights])
model.add(Conv1D(nb_filter=32, filter_length=3, border_mode= 'same' , activation= 'relu', input_length=10000))
model.add(MaxPooling1D(pool_length=2))
model.add(Flatten())
model.add(Dense(250, activation= 'relu' ))
model.add(Dense(4, activation= 'softmax' ))
model.compile(loss= 'categorical_crossentropy', optimizer= 'adam' , metrics=[ 'accuracy' ])
print(model.summary())
# Fit the model
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), nb_epoch=10, batch_size=128, verbose=1)
# Final evaluation of the model

scores = model.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

SyntaxError: invalid syntax (<ipython-input-27-4bcb2d88e5c5>, line 4)

In [24]:
# sentences = []
# for review in group_train_test_category.review:
#     review = review.split()
#     sentences.append(review)

# model['sentences'].shape
# word2vec = Word2Vec(sentences, workers=2)

#  model.wv.vocab.items()

# # model.wv.syn0


# embeddings = np.zeros((word2vec.wv.syn0.shape[0] + 1, word2vec.wv.syn0.shape[1]), dtype = "float32")



# embeddings[:model.wv.syn0.shape[0]] = word2vec.wv.syn0
# # embeddings.shape
# word2vec.wv.syn0.shape

# vocab = dict([(k, v.index) for k, v in model.wv.vocab.items()])

# model.most_similar('good')

